Creación de un dataset con la información de diferentes páginas web sobre las 10 mejores empresas españolas. 

In [1]:
import requests
#page = requests.get('https://contrataciondelestado.es/wps/portal/!ut/p/b1/jY5JDoJAFETP4gHM_z0qy2ZugoIKKL0hLIzBMGyM57c1bkVqV8l7qQID9ZoQKZAhYQ5cwIzts7u1j24a2_7djWx4kHleGFPcnpiPNPXLUsa2RsIC9RxAl_mCebxKqlyedISo49BPSyKsLpf5-CMK__lnMPMI_QJzFz_AzId9PA1XqC22aVQVHJR2GGbu0Q4l-a7II0oQORRQuzCYPgwdfeetWr0AY5oKyA!!/dl4/d5/L2dJQSEvUUt3QS80SmtFL1o2X0JTODhBQjFBMEdTTTEwQTZFMzY1MjAxR1My/')
#page = requests.get('https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/es_ES/search.do?reqCode=advancedSearch')
#page= requests.get('https://www.eltiempo.es/madrid.html')

respuesta1 = requests.get('https://ranking-empresas.eleconomista.es/')
respuesta2= requests.get('https://www.infocif.es/ranking/ventas-empresas/espana')
# Comprobamos que la petición al servidor ha resultado correcta -> código 200
print(respuesta1.status_code)
print(respuesta2.status_code)

from bs4 import BeautifulSoup
html1 = respuesta1.content
html2 = respuesta2.content
#Usamos BeautifulSoup para convertir el contenido descargado en un objeto de python que llamaremos soup que nos permite operar sobre él. 
soup1 = BeautifulSoup(html1,"html.parser")
soup2 = BeautifulSoup(html2,"html.parser")

# Fichero robots.txt en 
# https://ranking-empresas.eleconomista.es/robots.txt
# https://www.infocif.es/robots.txt


200
200


In [2]:
# Comenzamos haciendo webscrapping sobre la primera página web, extrayendo la información relevante.  
# Pasaremos la informaicón contenido en el un ranking de la página web a una dataframe. 
# Come veremos serán necesarias tareas de limpieza de los nombre de las variables

import pandas as pd
tabla= soup1.find("table", attrs={"class": "t_responsive1 txt_responsive1"})
ths=tabla.find("thead").find_all('th')
col_headings= [th.text.strip('\n') for th in ths]


## En esta sección limpiamos los nombres de las columnas: pasamos a minúsculas, eliminamos tildes, caractéres no alfanuméricos...
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
import re
from unicodedata import normalize 

# Creamos una función para poder aplicarla a cada uno de los nombres de las columnas
def cleaner_words (s):
  # Limpiamos de caracteres que no sean alfanuméricos
  clean_s = re.compile('[\W_]+')
  # Donde había un caracter no alfanumérico indicamos que se deje un espacio 
  words = re.sub(clean_s, ' ', s)
  #words = re.sub('^ ', '', s)
  # Para quitar las tildes hacemos uso de este código: Fuente https://es.stackoverflow.com/questions/135707/c%C3%B3mo-puedo-reemplazar-las-letras-con-tildes-por-las-mismas-sin-tilde-pero-no-l/135736
  words= re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",normalize( "NFD", words), 0, re.I)
  words = normalize( 'NFC', words)
  #Eliminamos los posibles espacios en blanco al principio de cada nombres de columna. Fuente https://j2logo.com/eliminar-espacios-en-blanco-string-strip-python/
  words=words.lstrip()
  # Vamos a reducir la longitud de los nombres de las columnas quedándonos solo con la primera palabra
  words= words.split(' ')
  words=words[0]
  # Pasamos a minúsculas
  return words.lower()

# Por medio de una list_comprenhension creamos la lista de los nombres de las columnas después de aplicar la función de limpieza de caracteres
col_headings = [cleaner_words (i) for i in col_headings]
print('columnas tabla empresas: ',col_headings)

#Obtenemos las filas que contienen las etiquetas con las información que queremos
trs=tabla.find_all('tr')

#Obtenemos las celdas por fila
tds= [tr.find_all('td') for tr in trs]

#Creamos una lista con el valor de las filas que hemos obtenido haciendo web scrapping
table=[]
for i in range(1,len(tds)):
  table.append([tt.text.strip('\n') for tt in tds[i]])

# Generaramos un dataframe que contiene la tabla que hemos 'scrapeado'
df_empresas_1=pd.DataFrame(table, columns=col_headings)
# Eliminamos la última fila porque contiene información que no corresponde a una fila del dataframe
df_empresas_1=df_empresas_1.drop(10,axis=0)
# Eliminamos la última columna -la número 7- que tampoco contiene información útil. Escribimos 6 porque se empieza a numerar desde el 0. 
df_empresas_1 = df_empresas_1.drop(df_empresas_1.columns[[6]], axis='columns')
# Eliminamos los puntos usados para marcar los datos de facturación, de este modo podremos hacer cáculos o representación gráficas sin que haya problemas. 
df_empresas_1['facturacion']=df_empresas_1['facturacion'].str.replace('[\W_]+', '')
# Algunas empresas, en ambos dataframes tienen SAU en uno y SA en otro, por este motivo homogeneizamos
df_empresas_1['nombre'] = df_empresas_1['nombre'].str.replace(' SAU', ' SA')
# Observamos el resultado final 
df_empresas_1


columnas tabla empresas:  ['posicionpos', 'evolucion', 'nombre', 'facturacion', 'sector', 'provincia', '']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\promo02\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\promo02\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,posicionpos,evolucion,nombre,facturacion,sector,provincia
0,1,0,MERCADONA SA,23343778000,4711,Valencia
1,2,0,REPSOL PETROLEO SA,21270559000,1920,Madrid
2,3,0,COMPAÑIA ESPAÑOLA DE PETROLEOS SA,18307333000,1920,Madrid
3,4,1,INDUSTRIA DE DISEÑO TEXTIL SA,18261000000,4642,Coruña
4,5,1,CEPSA TRADING SA,16494222000,4671,Madrid
5,6,0,REPSOL COMERCIAL DE PRODUCTOS PETROLIFEROS SA,15714171000,4671,Madrid
6,7,0,ENDESA ENERGIA SA,12603582000,3514,Madrid
7,8,0,EL CORTE INGLES SA,11374025000,4719,Madrid
8,9,0,SEAT SA,11157300000,2910,Barcelona
9,10,1,SOCIEDAD ESTATAL LOTERIAS Y APUESTAS DEL ESTAD...,9257347000,9200,Madrid


In [3]:
# Realizaremos la misma operación que en el ejercicio anterior con la segunda página web. En la generación de este dataframe tendremos que trabajar más sobre los valores sometiéndolos a un 
# proceso de limpieza para eliminar carácteres superfluos. 

import pandas as pd

tabla= soup2.find("table", attrs={"class": "display responsive nowrap"})

# En esta caso la obtención de los nombres de las columnas no ha planteado problemas y son adecaudos, aunque tengamos que modificarlos depués cuando construyamos el datafram conjunto.
ths=tabla.find("thead").find_all('th')
col_headings= [th.text.strip('\n') for th in ths]
# Por medio de una list_comprenhension quitamos los espacios en blanco al final de los nombres de las columnas. Fuente -> https://j2logo.com/eliminar-espacios-en-blanco-string-strip-python/
col_headings= [i.rstrip() for i in col_headings]
# Cambiamos a minúsculas los nombres de las columnas para mantener coherencia de criterio con el anterior dataframe
col_headings= [i.lower() for i in col_headings]
print('columnas tabla empresas: ',col_headings)

#Obtenermos las filas
trs=tabla.find_all('tr')

#Obtenemos las celdas por fila
tds= [tr.find_all('td') for tr in trs]

#Creamos una lista con el valor de las filas
table=[]
for i in range(1,len(tds)):
  table.append([tt.text.strip('\n') for tt in tds[i]])

#Generar el dataframe resultante que contiene la tabla
df_empresas_2=pd.DataFrame(table, columns=col_headings)

columns_for_cleaning= ['rank', 'ventas 2020', 'ventas 2019' , 'empleados', 'resultado 2020', 'ebitda 2020']
for i in columns_for_cleaning: 
  df_empresas_2[i]=df_empresas_2[i].str.replace('[\W_]+', '')
# Para eliminar los carácteres \n,\r,\t que se hayan en los valores de la columna 'Población (Provincia)' usamos el método replace -> https://www.delftstack.com/es/howto/python/remove-n-from-string-python/
df_empresas_2['población (provincia)'] = df_empresas_2['población (provincia)'].str.replace('\n', '')
df_empresas_2['población (provincia)'] = df_empresas_2['población (provincia)'].str.replace('\r', '')
df_empresas_2['población (provincia)'] = df_empresas_2['población (provincia)'].str.replace('\t', '')

# Ahora de los valores de esta columna queremos quedarnos solo con la población, de modo que tenemos que eliminar los paréntesis y la provincia contenida en ellos. 
df_empresas_2['población (provincia)'] = df_empresas_2['población (provincia)'].str.replace('\(.*\)', '')
# Convertimos a minúsculas los nombres de las ciudad y provincias para manetener coherencia con los valores del otro dataframe
df_empresas_2['población (provincia)'] = df_empresas_2['población (provincia)'].str.lower()
# Por último, cambiamos el nombre de la columna a población para ajustarse a los datos contenidos. 
df_empresas_2 = df_empresas_2.rename(columns={'población (provincia)' : 'poblacion'})

df_empresas_2['razón social'] = df_empresas_2['razón social'].str.replace(' SAU', ' SA')

# Visualizamos el dataframe resutlante
df_empresas_2

columnas tabla empresas:  ['rank', 'razón social', 'ventas 2020', 'ventas 2019', 'resultado 2020', 'ebitda 2020', 'empleados', 'sector actividad', 'población (provincia)', '']


,rank,razón social,ventas 2020,ventas 2019,resultado 2020,ebitda 2020,empleados,sector actividad,poblacion,
0,1,MERCADONA SA,24649629000,23446740000,727263000,1616007000,98141,Minoristas de alimentos / medicamentos,tavernes blanqu ...,
1,2,REPSOL PETROLEO SA,13422835000,21758602000,2415630000,565439000,3630,Petróleo y gas,madrid ...,
2,3,COMPAÑIA ESPAÑOLA DE PETROLEOS SA,11228832000,18320113000,815545000,841629000,2723,Petróleo y gas,madrid ...,
3,4,TELEFONICA SA,10807000000,4855000000,1685000000,944000000,1200,Telecomunicaciones,madrid ...,
4,5,ENDESA ENERGIA SA,10713848000,12652181000,816936000,1027956000,857,Utilities / Servicios de utilidad pública,madrid ...,
5,6,REPSOL COMERCIAL DE PRODUCTOS PETROLIFEROS SA,10362445000,15810828000,333054000,405190000,899,Petróleo y gas,madrid ...,
6,7,INDUSTRIA DE DISEÑO TEXTIL SA,10119000000,18499000000,2923000000,3022000000,,Ropa / textiles,arteixo ...,
7,8,GRUPO EDITORIAL BRUÑO SL,,9844000000,,,25,Edición y publicación,madrid ...,
8,9,EL CORTE INGLES SA,9461528000,11672424000,2402167000,2036911000,62749,Minoristas (excepto alimentos y medicamentos),madrid ...,
9,10,CEPSA TRADING SA,9158635000,16494222000,51063000,71075000,123,Petróleo y gas,madrid ...,


In [4]:
# Para unir los dos dataframes usamos una unión izquierda, tomando dataframe_1 con sus diez registros como la base, de este modo completamos la información de df_empresas_1 con los
# datos de df_empresas_2. Fuente -> https://datacarpentry.org/python-ecology-lesson-es/05-merging-data/

df_empresas_3= pd.merge(left=df_empresas_1,right=df_empresas_2, how='left', left_on='nombre', right_on='razón social')

# Eliminamos columnas cuya información tenemos duplicada
df_empresas_3 = df_empresas_3.drop(columns=['razón social'])
df_empresas_3 = df_empresas_3.drop(columns=['posicionpos'])

#Eliminamos el espacio en blanco en el valor de empleados de INDUSTRIA DE DISEÑO TEXTIL SA
df_empresas_3['empleados'] = df_empresas_3['empleados'].str.replace('\'', '')

# Reordenamos las columnas 
df_empresas_3 = df_empresas_3.reindex(columns=['nombre','rank', 'poblacion', 'provincia' , 'sector' , 'sector actividad', 'empleados', 'evolucion', 'facturacion' ,'ventas 2019','ventas 2020','resultado 2020', 'ebitda 2020'])

#Visualizamos los resultados
df_empresas_3


,nombre,rank,poblacion,provincia,sector,sector actividad,empleados,evolucion,facturacion,ventas 2019,ventas 2020,resultado 2020,ebitda 2020
0,MERCADONA SA,1,tavernes blanqu ...,Valencia,4711,Minoristas de alimentos / medicamentos,98141,0,23343778000,23446740000,24649629000,727263000,1616007000
1,REPSOL PETROLEO SA,2,madrid ...,Madrid,1920,Petróleo y gas,3630,0,21270559000,21758602000,13422835000,2415630000,565439000
2,COMPAÑIA ESPAÑOLA DE PETROLEOS SA,3,madrid ...,Madrid,1920,Petróleo y gas,2723,0,18307333000,18320113000,11228832000,815545000,841629000
3,INDUSTRIA DE DISEÑO TEXTIL SA,7,arteixo ...,Coruña,4642,Ropa / textiles,,1,18261000000,18499000000,10119000000,2923000000,3022000000
4,CEPSA TRADING SA,10,madrid ...,Madrid,4671,Petróleo y gas,123,1,16494222000,16494222000,9158635000,51063000,71075000
5,REPSOL COMERCIAL DE PRODUCTOS PETROLIFEROS SA,6,madrid ...,Madrid,4671,Petróleo y gas,899,0,15714171000,15810828000,10362445000,333054000,405190000
6,ENDESA ENERGIA SA,5,madrid ...,Madrid,3514,Utilities / Servicios de utilidad pública,857,0,12603582000,12652181000,10713848000,816936000,1027956000
7,EL CORTE INGLES SA,9,madrid ...,Madrid,4719,Minoristas (excepto alimentos y medicamentos),62749,0,11374025000,11672424000,9461528000,2402167000,2036911000
8,SEAT SA,11,martorell ...,Barcelona,2910,Automotor,195,0,11157300000,11423000000,8999200000,194200000,101200000
9,SOCIEDAD ESTATAL LOTERIAS Y APUESTAS DEL ESTAD...,16,madrid ...,Madrid,9200,Productos y actividades relacionados con ocio ...,506,1,9257347000,9310445000,7735662000,1455990000,1947681000


In [5]:
 # Estudio de los componentes del dataframe
 print ('Número de filas : %d' % (df_empresas_3.shape[0])) 
 print ('Número de columnas : %d' % (df_empresas_3.shape[1])) 

# Información de los tipos de objetos de cada variable
df_empresas_3.info()


Número de filas : 10
Número de columnas : 13
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   nombre            10 non-null     object
 1   rank              10 non-null     object
 2   poblacion         10 non-null     object
 3   provincia         10 non-null     object
 4   sector            10 non-null     object
 5   sector actividad  10 non-null     object
 6   empleados         10 non-null     object
 7   evolucion         10 non-null     object
 8   facturacion       10 non-null     object
 9   ventas 2019       10 non-null     object
 10  ventas 2020       10 non-null     object
 11  resultado 2020    10 non-null     object
 12  ebitda 2020       10 non-null     object
dtypes: object(13)
memory usage: 1.1+ KB


In [6]:
# Como podemos observar hay una serie de columnas cuyos tipos en python no se corresponden con el tipo real, por ellos deberemos convertir a tipos numéricos las
# siguientes columnas, para poder hacer cálculos con ellas (porcentajes, medias, etc...). 

columnas_numericas =['empleados', 'facturacion' , 'ventas 2019', 'ventas 2020', 'resultado 2020', 'ebitda 2020']
for i in columnas_numericas:
    df_empresas_3[i] = pd.to_numeric(df_empresas_3[i], downcast='integer')

# Comprobamos que los cambios de han realizado
df_empresas_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nombre            10 non-null     object 
 1   rank              10 non-null     object 
 2   poblacion         10 non-null     object 
 3   provincia         10 non-null     object 
 4   sector            10 non-null     object 
 5   sector actividad  10 non-null     object 
 6   empleados         9 non-null      float64
 7   evolucion         10 non-null     object 
 8   facturacion       10 non-null     int64  
 9   ventas 2019       10 non-null     int64  
 10  ventas 2020       10 non-null     int64  
 11  resultado 2020    10 non-null     int64  
 12  ebitda 2020       10 non-null     int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 1.1+ KB


In [11]:
# Exportamos nuestro dataframe a un archivo csv. Como no queremos exportar el índice indicamos index=False
df_empresas_3.to_csv(r'C:\Users\promo02\Desktop\Master\Tipología y ciclo de vida de los datos\PRAC1\dataset_empresas_españolas.csv', 
                     index=False, 
                     sep = ';',
                    encoding='utf-8-sig')